In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Model

# Data

In [ ]:
import pathlib
dataset = pathlib.Path(r"C:\Users\lulu5\Documents\echantillon_1")
image_count = len(list(dataset.glob('*/*.jpg')))
print(image_count)

In [ ]:
batch_size = 128
img_height = 224
img_width = 224
number_class = 1081

In [ ]:
TFs = {#'height_shift_range':  .5,
       "horizontal_flip": True,
       "vertical_flip": True,
       "rotation_range": 30,
       "featurewise_std_normalization": True,
       "brightness_range": (1, 1.5),
       "shear_range":0.2
       # other transformation your want
       # ...
       }
#datagen = ImageDataGenerator(**TFs, validation_split=0.2)
datagen = ImageDataGenerator(**TFs, validation_split=0.2)

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  dataset,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  dataset,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


train_generator = datagen.flow_from_directory(
    dataset,
    color_mode="rgb",
    batch_size=batch_size,
    subset = "training",
    target_size=(img_height, img_width),
    shuffle=True,
    class_mode='binary')

validation_generator = datagen.flow_from_directory(
    dataset,
    color_mode="rgb",
    batch_size=batch_size,
    subset = "validation",
    target_size=(img_height, img_width),
    shuffle=True,
    class_mode='binary')

In [ ]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

In [ ]:
class_names = train_ds.class_names

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
epochs = 10

# Optimisation

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
model_V1_augmented = Sequential()
model_V1_augmented.add(data_augmentation)
model = tf.keras.applications.resnet50.ResNet50(include_top=True, weights=None, input_tensor=None,input_shape=None, pooling=None, classes=number_class)
extracted_layer= Model(inputs = model.input, outputs=model.get_layer("avg_pool").output, name="model_V1")
extracted_layer.trainable = False
model_V1_augmented.add(extracted_layer)

In [ ]:
model.compile(optimizer="Adam", 
        loss=SparseCategoricalFocalLoss(gamma=5),
        metrics=['accuracy', 'sparse_top_k_categorical_accuracy'])

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[cp_callback]
)

In [ ]:
#model_V1_augmented.save_weights(checkpoint_path.format(epoch=0))

In [ ]:
#model_V1_augmented.summary()

In [ ]:
from focal_loss import SparseCategoricalFocalLoss

In [ ]:
model.compile(optimizer="Adam", 
        loss=SparseCategoricalFocalLoss(gamma=5),
        metrics=['accuracy', 'sparse_top_k_categorical_accuracy'])

In [ ]:

epochs = 10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[cp_callback]
)
*/

# Unoptimized

In [ ]:
model_unoptimized = tf.keras.applications.resnet50.ResNet50(
    include_top=True, weights=None, input_tensor=None,
    input_shape=None, pooling=None, classes=number_class)

model_unoptimized.compile(optimizer="Adam", 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy', 'sparse_top_k_categorical_accuracy'])

model_unoptimized.fit(train_generator, validation_data=validation_generator,epochs=epochs,batch_size=batch_size, verbose=1,
steps_per_epoch=3, validation_steps=1)

In [ ]:
model_unoptimized.compile(optimizer="Adam", 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy', 'sparse_top_k_categorical_accuracy'])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()